# NarrativeVerse: Branching Story Generation

This notebook demonstrates how to create dynamic, branching narratives using the NarrativeVerse framework. We'll explore how to craft a story that adapts to player choices, creating a personalized adventure experience.

Our example will follow a light, uplifting adventure centered around discovery, teamwork, and positive choices. The story begins with a cooperative mission to restore an ancient energy source and branches into multiple creative paths featuring unexpected friendships, symbolic rewards, and imaginative challenges.

## Setup and Imports

In [1]:
import json
import random
from typing import Dict, List, Optional, Tuple, Any
import sys
import os

# Add the parent directory to the path so we can import the manus_engine module
sys.path.append('..')
from manus_engine import ManusEngine

## Story Structure

A branching narrative can be represented as a directed graph, where:
- Nodes represent story segments or scenes
- Edges represent choices or transitions between segments
- Each node can have multiple outgoing edges (choices)
- The player's path through this graph creates a unique story experience

Let's define a simple structure for our story nodes:

In [2]:
class StoryNode:
    """Represents a single node in our branching story."""
    
    def __init__(self, 
                 node_id: str,
                 title: str,
                 content: str,
                 choices: Optional[List[Dict]] = None,
                 npc_interactions: Optional[List[Dict]] = None,
                 is_ending: bool = False):
        """
        Initialize a story node.
        
        Args:
            node_id: Unique identifier for this node
            title: Title or heading for this story segment
            content: Main narrative text for this segment
            choices: List of possible choices, each with 'text' and 'next_node' fields
            npc_interactions: List of NPC interactions in this node
            is_ending: Whether this node is a story ending
        """
        self.node_id = node_id
        self.title = title
        self.content = content
        self.choices = choices or []
        self.npc_interactions = npc_interactions or []
        self.is_ending = is_ending
    
    def add_choice(self, text: str, next_node: str, impact: Optional[Dict] = None):
        """Add a choice option to this node."""
        self.choices.append({
            'text': text,
            'next_node': next_node,
            'impact': impact or {}
        })
    
    def add_npc_interaction(self, npc_name: str, context: str, dialogue_key: str):
        """Add an NPC interaction to this node."""
        self.npc_interactions.append({
            'npc_name': npc_name,
            'context': context,
            'dialogue_key': dialogue_key
        })

## Story Graph

Now, let's create a class to manage our entire story structure:

In [3]:
class StoryGraph:
    """Manages the entire branching story structure."""
    
    def __init__(self, title: str, description: str):
        """
        Initialize a new story graph.
        
        Args:
            title: The title of the story
            description: A brief description of the story
        """
        self.title = title
        self.description = description
        self.nodes = {}
        self.start_node = None
        self.player_state = {
            'inventory': [],
            'relationships': {},
            'choices_made': [],
            'flags': {}
        }
    
    def add_node(self, node: StoryNode):
        """Add a node to the story graph."""
        self.nodes[node.node_id] = node
        # If this is the first node, set it as the start node
        if not self.start_node:
            self.start_node = node.node_id
    
    def get_node(self, node_id: str) -> Optional[StoryNode]:
        """Get a node by its ID."""
        return self.nodes.get(node_id)
    
    def make_choice(self, current_node_id: str, choice_index: int) -> str:
        """Make a choice and return the next node ID."""
        current_node = self.get_node(current_node_id)
        if not current_node or choice_index >= len(current_node.choices):
            return current_node_id  # Stay on the same node if invalid
        
        choice = current_node.choices[choice_index]
        self.player_state['choices_made'].append({
            'node': current_node_id,
            'choice': choice['text']
        })
        
        # Apply any impacts from this choice
        if 'impact' in choice and choice['impact']:
            self._apply_choice_impact(choice['impact'])
        
        return choice['next_node']
    
    def _apply_choice_impact(self, impact: Dict):
        """Apply the impact of a choice to the player state."""
        # Add items to inventory
        if 'add_inventory' in impact:
            for item in impact['add_inventory']:
                if item not in self.player_state['inventory']:
                    self.player_state['inventory'].append(item)
        
        # Remove items from inventory
        if 'remove_inventory' in impact:
            for item in impact['remove_inventory']:
                if item in self.player_state['inventory']:
                    self.player_state['inventory'].remove(item)
        
        # Update relationship values
        if 'relationships' in impact:
            for npc, change in impact['relationships'].items():
                if npc not in self.player_state['relationships']:
                    self.player_state['relationships'][npc] = 50  # Default starting value
                self.player_state['relationships'][npc] += change
                # Ensure relationship stays within bounds
                self.player_state['relationships'][npc] = max(0, min(100, self.player_state['relationships'][npc]))
        
        # Set flags
        if 'flags' in impact:
            for flag, value in impact['flags'].items():
                self.player_state['flags'][flag] = value
    
    def reset_player_state(self):
        """Reset the player state to its initial values."""
        self.player_state = {
            'inventory': [],
            'relationships': {},
            'choices_made': [],
            'flags': {}
        }
    
    def to_dict(self) -> Dict:
        """Convert the story graph to a dictionary for serialization."""
        return {
            'title': self.title,
            'description': self.description,
            'start_node': self.start_node,
            'nodes': {
                node_id: {
                    'title': node.title,
                    'content': node.content,
                    'choices': node.choices,
                    'npc_interactions': node.npc_interactions,
                    'is_ending': node.is_ending
                } for node_id, node in self.nodes.items()
            }
        }
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'StoryGraph':
        """Create a story graph from a dictionary."""
        graph = cls(data['title'], data['description'])
        graph.start_node = data['start_node']
        
        for node_id, node_data in data['nodes'].items():
            node = StoryNode(
                node_id=node_id,
                title=node_data['title'],
                content=node_data['content'],
                choices=node_data.get('choices', []),
                npc_interactions=node_data.get('npc_interactions', []),
                is_ending=node_data.get('is_ending', False)
            )
            graph.nodes[node_id] = node
        
        return graph

## Story Runner

Now, let's create a class to run our story and handle NPC interactions:

In [4]:
class StoryRunner:
    """Runs a branching story with NPC interactions."""
    
    def __init__(self, story_graph: StoryGraph, manus_engine: Optional[ManusEngine] = None):
        """
        Initialize a story runner.
        
        Args:
            story_graph: The story graph to run
            manus_engine: Optional ManusEngine instance for NPC dialogue generation
        """
        self.story_graph = story_graph
        self.current_node_id = story_graph.start_node
        self.manus_engine = manus_engine or ManusEngine()
        self.dialogue_cache = {}
    
    def start(self):
        """Start the story from the beginning."""
        self.story_graph.reset_player_state()
        self.current_node_id = self.story_graph.start_node
        return self.get_current_node()
    
    def get_current_node(self) -> Optional[StoryNode]:
        """Get the current story node."""
        return self.story_graph.get_node(self.current_node_id)
    
    def make_choice(self, choice_index: int) -> StoryNode:
        """Make a choice and advance to the next node."""
        self.current_node_id = self.story_graph.make_choice(self.current_node_id, choice_index)
        return self.get_current_node()
    
    def get_npc_dialogue(self, npc_name: str, context: str, dialogue_key: str = None) -> str:
        """Get dialogue for an NPC in the current context."""
        # Check if we have this dialogue cached
        cache_key = f"{npc_name}:{context}:{dialogue_key}"
        if cache_key in self.dialogue_cache:
            return self.dialogue_cache[cache_key]
        
        # Generate new dialogue
        dialogue = self.manus_engine.generate_dialogue(
            npc_name=npc_name,
            context=context,
            player_history=self.story_graph.player_state
        )
        
        # Cache the dialogue
        self.dialogue_cache[cache_key] = dialogue
        return dialogue
    
    def process_npc_interactions(self) -> List[Dict]:
        """Process all NPC interactions in the current node."""
        current_node = self.get_current_node()
        if not current_node:
            return []
        
        interactions = []
        for interaction in current_node.npc_interactions:
            npc_name = interaction['npc_name']
            context = interaction['context']
            dialogue_key = interaction.get('dialogue_key')
            
            dialogue = self.get_npc_dialogue(npc_name, context, dialogue_key)
            interactions.append({
                'npc_name': npc_name,
                'dialogue': dialogue
            })
        
        return interactions
    
    def display_current_node(self):
        """Display the current node with NPC interactions."""
        current_node = self.get_current_node()
        if not current_node:
            print("Error: No current node found.")
            return
        
        print(f"\n=== {current_node.title} ===\n")
        print(current_node.content)
        
        # Process NPC interactions
        interactions = self.process_npc_interactions()
        if interactions:
            print("\n--- Character Dialogue ---")
            for interaction in interactions:
                print(f"\n{interaction['npc_name']}: \"{interaction['dialogue']}\"")
        
        # Display choices
        if current_node.choices and not current_node.is_ending:
            print("\n--- Your Choices ---")
            for i, choice in enumerate(current_node.choices):
                print(f"[{i+1}] {choice['text']}")
        elif current_node.is_ending:
            print("\n--- THE END ---")
            print("This branch of the story has concluded.")
    
    def interactive_run(self):
        """Run the story interactively in the console."""
        self.start()
        
        while True:
            self.display_current_node()
            current_node = self.get_current_node()
            
            if current_node.is_ending or not current_node.choices:
                break
            
            # Get player choice
            valid_choice = False
            while not valid_choice:
                try:
                    choice = int(input("\nEnter your choice (number): "))
                    if 1 <= choice <= len(current_node.choices):
                        valid_choice = True
                    else:
                        print(f"Please enter a number between 1 and {len(current_node.choices)}.")
                except ValueError:
                    print("Please enter a valid number.")
            
            # Make the choice (adjust for 0-based indexing)
            self.make_choice(choice - 1)

## Creating Our Adventure Story

Now, let's create our uplifting adventure story about restoring an ancient energy source. We'll include multiple branches, unexpected friendships, and positive choices.

In [5]:
def create_luminous_source_adventure() -> StoryGraph:
    """Create our adventure story about restoring the Luminous Source."""
    
    story = StoryGraph(
        title="The Luminous Source",
        description="An uplifting adventure to restore an ancient energy source that once brought harmony to the world."
    )
    
    # Create story nodes
    
    # Introduction
    intro = StoryNode(
        node_id="intro",
        title="The Call to Adventure",
        content="The floating city of Aetheria has been slowly losing power for months. The ancient Luminous Source that once provided endless clean energy has been fading, and no one knows why. As a respected problem-solver with a knack for understanding old technology, you've been summoned by the Council of Sages to join a special expedition team. Their mission: journey to the heart of the Prismatic Mountains, discover what's affecting the Luminous Source, and restore it before Aetheria's systems fail completely."
    )
    intro.add_npc_interaction("Captain Lyra Novastella", "greeting", "intro_meeting")
    intro.add_choice("Accept the mission with enthusiasm", "team_assembly", {
        'relationships': {'Captain Lyra Novastella': 10}
    })
    intro.add_choice("Ask for more details before committing", "mission_briefing", {
        'flags': {'cautious_approach': True}
    })
    story.add_node(intro)
    
    # Mission Briefing (if player asks for more details)
    briefing = StoryNode(
        node_id="mission_briefing",
        title="The Mission Briefing",
        content="The Council's chief scientist, Dr. Elian Thaumatec, steps forward with a holographic display showing the Luminous Source's energy output over the past year. The decline is clear and accelerating. 'We've tried remote diagnostics and sending drones, but something in the mountains disrupts our signals. This has to be an in-person expedition,' he explains. 'The journey will be challenging, but we've assembled the finest team possible. Your expertise with ancient systems could be the key to saving our city.'"
    )
    briefing.add_npc_interaction("Dr. Elian Thaumatec", "quest_offer", "mission_explanation")
    briefing.add_choice("Accept the mission now that you understand the stakes", "team_assembly", {
        'relationships': {'Dr. Elian Thaumatec': 5},
        'flags': {'well_informed': True}
    })
    story.add_node(briefing)
    
    # Team Assembly
    team = StoryNode(
        node_id="team_assembly",
        title="Meeting Your Team",
        content="You're introduced to the expedition team at Aetheria's skyport. Captain Lyra Novastella, a renowned explorer with an uncanny sense for navigation, will lead the mission. Dr. Elian Thaumatec, the brilliant but eccentric scientist, brings his experimental equipment for analyzing the Luminous Source. Sylva Aerafrond, Guardian of the Floating Forests, offers her knowledge of the natural world and ability to create safe passages through difficult terrain. A small, curious creature named Blip bounces excitedly around the group—apparently Lyra's alien companion who has a knack for sensing energy patterns. The final member is Mira Lumina, a young apprentice eager to prove herself on her first major expedition."
    )
    team.add_npc_interaction("Sylva Aerafrond", "greeting", "team_meeting")
    team.add_npc_interaction("Blip", "greeting", "team_meeting")
    team.add_choice("Suggest taking the direct route through Crystal Canyon", "crystal_canyon", {
        'flags': {'route_choice': 'direct'}
    })
    team.add_choice("Recommend the longer but potentially safer Sky River path", "sky_river", {
        'flags': {'route_choice': 'safe'},
        'relationships': {'Sylva Aerafrond': 5}
    })
    story.add_node(team)
    
    # Crystal Canyon Route
    canyon = StoryNode(
        node_id="crystal_canyon",
        title="The Crystal Canyon",
        content="Your team descends into the breathtaking Crystal Canyon, where massive formations of luminescent crystals create natural light shows on the rock walls. The direct route seems promising until you encounter a massive ravine where the path once was. A recent avalanche has destroyed the old crossing. As everyone considers turning back, Blip begins bouncing excitedly, pointing to strange geometric patterns etched into nearby crystals."
    )
    canyon.add_npc_interaction("Blip", "discovery", "crystal_patterns")
    canyon.add_choice("Examine the crystal patterns Blip is indicating", "crystal_puzzle", {
        'relationships': {'Blip': 10}
    })
    canyon.add_choice("Ask Sylva if she can grow a natural bridge across the ravine", "living_bridge", {
        'relationships': {'Sylva Aerafrond': 10}
    })
    story.add_node(canyon)
    
    # Crystal Puzzle
    puzzle = StoryNode(
        node_id="crystal_puzzle",
        title="The Ancient Mechanism",
        content="Dr. Thaumatec helps you analyze the crystal patterns, which appear to be an ancient control mechanism. 'This is fascinating! These crystals are quantum-entangled with something across the ravine,' he explains excitedly. Working together, your team deciphers the sequence needed to activate the mechanism. When the final crystal is touched in sequence, a dazzling bridge of pure light materializes across the ravine. 'The ancients who built the Luminous Source must have created this as a security measure,' Lyra observes. 'Only those who understand their technology can pass.'"
    )
    puzzle.add_npc_interaction("Dr. Elian Thaumatec", "teaching_moments", "crystal_explanation")
    puzzle.add_choice("Cross the light bridge and continue toward the Luminous Source", "energy_creatures", {
        'inventory': ['Crystal Activation Sequence'],
        'flags': {'solved_crystal_puzzle': True}
    })
    story.add_node(puzzle)
    
    # Living Bridge
    bridge = StoryNode(
        node_id="living_bridge",
        title="Sylva's Living Bridge",
        content="Sylva steps forward, her expression serene. 'I can help, but I'll need everyone's assistance.' She produces a handful of glowing seeds from a pouch at her belt. 'These are Skyroot seedlings. They grow toward energy sources.' Following her instructions, the team plants the seeds along the edge of the ravine. Sylva begins to sing a gentle melody, and the seeds respond immediately, sprouting and growing at an accelerated rate. Massive roots extend across the chasm, intertwining to form a sturdy natural bridge. 'The bridge is alive,' Sylva explains. 'It will hold us safely, but we must cross with respect.'"
    )
    bridge.add_npc_interaction("Sylva Aerafrond", "teaching_moments", "nature_harmony")
    bridge.add_choice("Cross the living bridge with gratitude and continue the journey", "energy_creatures", {
        'inventory': ['Skyroot Seedling'],
        'flags': {'created_living_bridge': True}
    })
    story.add_node(bridge)
    
    # Sky River Route
    river = StoryNode(
        node_id="sky_river",
        title="The Sky River Path",
        content="Your team takes the winding Sky River path, which follows an ancient waterway that flows impossibly upward along the mountainside. The route is longer but offers breathtaking views of floating water droplets that catch the light like thousands of prisms. As you round a bend, you encounter a young explorer sitting dejectedly beside a damaged glider. She introduces herself as Zephyr and explains that she was mapping the mountains when a sudden wind gust damaged her glider's propulsion system."
    )
    river.add_choice("Offer to help repair Zephyr's glider", "repair_glider", {
        'flags': {'helping_zephyr': True}
    })
    river.add_choice("Invite Zephyr to join your expedition temporarily", "zephyr_joins", {
        'flags': {'zephyr_joined': True}
    })
    story.add_node(river)
    
    # Repair Glider
    repair = StoryNode(
        node_id="repair_glider",
        title="Collaborative Repair",
        content="Dr. Thaumatec examines the glider's propulsion system while you and Mira help diagnose the problem. Working together, you identify a misaligned energy crystal that's disrupting the flow. With careful adjustments and some creative problem-solving, you manage to repair the glider. Zephyr is overjoyed and insists on repaying your kindness. 'I've been mapping these mountains for months,' she says, unfolding a detailed chart. 'This map shows a hidden shortcut that will save you half a day's journey. There's also a marker for some unusual energy readings I couldn't explain—might be related to your Luminous Source.'"
    )
    repair.add_npc_interaction("Mira Lumina", "teaching_moments", "teamwork_lesson")
    repair.add_choice("Thank Zephyr and follow her map to the shortcut", "mountain_heart", {
        'inventory': ['Detailed Mountain Map'],
        'flags': {'have_map': True}
    })
    story.add_node(repair)
    
    # Zephyr Joins
    joins = StoryNode(
        node_id="zephyr_joins",
        title="An Unexpected Ally",
        content="'Join you? Really?' Zephyr's face lights up with excitement. 'I'd be honored! I've been exploring these mountains for months.' She shoulders her pack and secures her damaged glider. 'I can't fly with this, but I know these paths better than anyone.' As you continue along the Sky River path, Zephyr points out features you might have missed—hidden caves, rare crystal formations, and signs of the ancient civilization that built the Luminous Source. Her knowledge proves invaluable when you reach a fork in the path that wasn't on your original maps."
    )
    joins.add_choice("Follow Zephyr's suggestion to take the higher path through the Whispering Caves", "whispering_caves", {
        'relationships': {'Captain Lyra Novastella': 5},
        'flags': {'zephyr_guidance': True}
    })
    story.add_node(joins)
    
    # Whispering Caves
    caves = StoryNode(
        node_id="whispering_caves",
        title="The Whispering Caves",
        content="The Whispering Caves are a marvel of natural acoustics. Every sound—from footsteps to whispers—creates musical echoes that resonate throughout the cavern system. Zephyr explains that local legends say the caves were used by the ancients for meditation and harmonizing with the Luminous Source's energy. As your team moves deeper into the caves, Blip becomes increasingly animated, bouncing from crystal to crystal and changing colors rapidly. Suddenly, the alien pet stops and emits a soft, melodic tone that causes all the crystals in the cave to resonate in harmony."
    )
    caves.add_npc_interaction("Blip", "discovery", "harmonic_resonance")
    caves.add_choice("Encourage everyone to join Blip in creating harmonic sounds", "harmonic_key", {
        'relationships': {'Blip': 15},
        'flags': {'discovered_harmonic_key': True}
    })
    story.add_node(caves)
    
    # Harmonic Key
    harmonic = StoryNode(
        node_id="harmonic_key",
        title="The Harmonic Key",
        content="Following your suggestion, the team begins creating sounds—Lyra whistles a melody, Dr. Thaumatec taps rhythmically on his equipment, Sylva hums in a low, resonant tone, and Mira sings a clear note. The combined sounds join with Blip's melody, and the cave comes alive with pulsing light. A hidden doorway materializes in the cave wall, revealing a passage that, according to Zephyr's amazement, wasn't on any known map. 'The ancients protected their most important paths with harmonic keys,' Dr. Thaumatec theorizes. 'Only those who could work together in harmony could proceed.'"
    )
    harmonic.add_choice("Enter the hidden passage", "mountain_heart", {
        'inventory': ['Harmonic Sequence'],
        'flags': {'used_harmonic_key': True}
    })
    story.add_node(harmonic)
    
    # Energy Creatures (from Crystal Canyon route)
    creatures = StoryNode(
        node_id="energy_creatures",
        title="The Light Dancers",
        content="As your team ascends higher into the Prismatic Mountains, the air becomes charged with energy. Suddenly, Blip freezes, then begins to glow brightly. Around you, the air shimmers as dozens of translucent, luminous beings materialize—like living light in vaguely humanoid form. They dance around your group, curious but cautious. 'Light Dancers,' Sylva whispers in awe. 'I thought they were just legends. They're said to be guardians of powerful energy sources, manifestations of the energy itself.'"
    )
    creatures.add_npc_interaction("Sylva Aerafrond", "discovery", "light_dancers_lore")
    creatures.add_choice("Attempt to communicate peacefully with the Light Dancers", "dancer_communication", {
        'flags': {'peaceful_approach': True}
    })
    creatures.add_choice("Ask Dr. Thaumatec to analyze them with his equipment", "dancer_analysis", {
        'relationships': {'Dr. Elian Thaumatec': 10},
        'flags': {'scientific_approach': True}
    })
    story.add_node(creatures)
    
    # Dancer Communication
    communication = StoryNode(
        node_id="dancer_communication",
        title="A Dance of Understanding",
        content="Instinctively, you open your hands in a gesture of peace and bow slightly to the Light Dancers. They respond by circling closer, their movements becoming more deliberate. Blip bounces forward and begins to move in patterns that mimic the Dancers. Understanding dawns on you—this is their language. Following Blip's example, your team begins to move in simple, harmonious patterns. The Light Dancers respond enthusiastically, and one approaches to touch your outstretched hand. A flood of images fills your mind: the Luminous Source, a disruption in its core, and a sense of fading, of diminishing."
    )
    communication.add_choice("Share the vision with your team and ask the Light Dancers to guide you", "dancer_guidance", {
        'relationships': {'Blip': 10, 'Sylva Aerafrond': 10},
        'flags': {'dancer_alliance': True}
    })
    story.add_node(communication)
    
    # Dancer Analysis
    analysis = StoryNode(
        node_id="dancer_analysis",
        title="Scientific Discovery",
        content="Dr. Thaumatec carefully activates his analysis equipment, adjusting it to its lowest power setting to avoid disturbing the Light Dancers. 'Fascinating!' he exclaims softly. 'They're composed of the same energy signature as the Luminous Source, but in a more evolved state. They're not just guardians—they're like its immune system.' The Light Dancers seem curious about the equipment, gathering around to observe. One of them extends a tendril of light toward the device, and the displays suddenly fill with complex data patterns. 'They're communicating!' Dr. Thaumatec gasps. 'They're showing us what's wrong with the Source!'"
    )
    analysis.add_npc_interaction("Dr. Elian Thaumatec", "discovery", "energy_analysis")
    analysis.add_choice("Ask the Light Dancers to guide you to the Luminous Source", "dancer_guidance", {
        'inventory': ['Energy Signature Data'],
        'flags': {'have_energy_data': True}
    })
    story.add_node(analysis)
    
    # Dancer Guidance
    guidance = StoryNode(
        node_id="dancer_guidance",
        title="Luminous Guides",
        content="The Light Dancers form a procession around your team, illuminating a path that winds up through increasingly rugged terrain. They move with purpose, occasionally pausing to ensure you're keeping pace. After several hours of climbing, you emerge onto a plateau overlooking a vast crater. In the center stands a towering spire of crystal and light—the Luminous Source. Even from a distance, you can see that parts of it are flickering and dim. The Light Dancers gesture urgently, indicating a narrow path down into the crater."
    )
    guidance.add_choice("Follow the Light Dancers down to the Luminous Source", "source_base", {
        'flags': {'guided_by_dancers': True}
    })
    story.add_node(guidance)
    
    # Mountain Heart (from Sky River route)
    heart = StoryNode(
        node_id="mountain_heart",
        title="The Heart of the Mountain",
        content="Your path leads to an immense cavern at the heart of the mountain. Massive crystal formations jut from the walls and ceiling, and a natural skylight allows sunbeams to filter through, creating rainbow patterns across the chamber. In the center of the cavern is a pool of water so still it looks like glass, perfectly reflecting the crystals above. Mira approaches the pool, fascinated. 'Look!' she calls out. 'There's something beneath the surface!' Sure enough, through the crystal-clear water, you can see an ornate platform with ancient markings, and beyond it, a tunnel that seems to glow with inner light."
    )
    heart.add_npc_interaction("Mira Lumina", "discovery", "ancient_platform")
    heart.add_choice("Carefully wade into the pool to examine the platform", "water_passage", {
        'flags': {'explored_pool': True}
    })
    heart.add_choice("Look for another way to access what's beneath the pool", "mechanism_search", {
        'flags': {'cautious_exploration': True}
    })
    story.add_node(heart)
    
    # Water Passage
    water = StoryNode(
        node_id="water_passage",
        title="The Reflecting Pool",
        content="You carefully step into the pool, expecting cold water, but instead find it pleasantly warm. The water barely ripples around your legs, maintaining its mirror-like surface. As you reach the central platform, you notice that the ancient markings form a map—one that matches the shape of the Prismatic Mountains themselves, with the Luminous Source clearly marked at the center. When your shadow falls across the map, the platform begins to descend slowly, carrying you down into a hidden chamber. The water somehow stays above, forming an impossible ceiling that continues to let light through."
    )
    water.add_choice("Signal your team to join you on the descending platform", "hidden_chamber", {
        'inventory': ['Ancient Mountain Map'],
        'flags': {'found_hidden_passage': True}
    })
    story.add_node(water)
    
    # Mechanism Search
    mechanism = StoryNode(
        node_id="mechanism_search",
        title="The Ancient Controls",
        content="Rather than entering the water, you circle the pool's edge, examining the surrounding cavern walls. Dr. Thaumatec joins you, running his fingers along the crystal formations. 'These aren't random,' he murmurs. 'They're arranged in patterns.' Together, you identify a series of crystals that seem out of alignment with the others. When Dr. Thaumatec carefully adjusts one, it clicks into place and begins to glow. 'It's a combination lock,' he realizes. Working methodically, your team aligns all the crystals, and a rumbling sound fills the cavern. The pool's water parts in the middle, revealing a staircase leading down to the platform and tunnel."
    )
    mechanism.add_npc_interaction("Dr. Elian Thaumatec", "problem_solving", "crystal_mechanism")
    mechanism.add_choice("Descend the stairs to the revealed platform", "hidden_chamber", {
        'inventory': ['Crystal Alignment Sequence'],
        'flags': {'solved_water_puzzle': True}
    })
    story.add_node(mechanism)
    
    # Hidden Chamber
    chamber = StoryNode(
        node_id="hidden_chamber",
        title="The Ancients' Transit System",
        content="The platform or stairs lead to a magnificent underground chamber lined with the same crystal technology you've encountered throughout your journey. At the center sits an elegant vessel—something between a boat and a hovering craft—adorned with the same markings as the platform above. Captain Lyra examines it with expert eyes. 'It's a transit system,' she concludes. 'The ancients built this to quickly reach the Luminous Source from different points in the mountains.' The tunnel beyond pulses with soft light, seeming to invite you forward. 'This will take us directly to the Source,' Lyra says confidently."
    )
    chamber.add_npc_interaction("Captain Lyra Novastella", "discovery", "ancient_transit")
    chamber.add_choice("Board the vessel and journey to the Luminous Source", "source_base", {
        'flags': {'used_transit_system': True}
    })
    story.add_node(chamber)
    
    # Source Base - All paths converge here
    source = StoryNode(
        node_id="source_base",
        title="The Base of the Luminous Source",
        content="Whether guided by the Light Dancers or arriving via the ancient transit system, your team finally reaches the base of the Luminous Source. Up close, the massive crystal spire is even more impressive—and its problems more apparent. Sections of the once-brilliant structure are dark or flickering weakly. The ground around the base is littered with crystal fragments that have fallen from above. Dr. Thaumatec's instruments go wild as he takes readings. 'The energy flow is disrupted,' he explains. 'Something is blocking the Source from drawing power from the earth and sky as it was designed to do.'"
    )
    source.add_npc_interaction("Dr. Elian Thaumatec", "problem_solving", "source_diagnosis")
    source.add_choice("Search for physical blockages or damage around the Source's base", "physical_repair", {
        'flags': {'practical_approach': True}
    })
    source.add_choice("Ask Sylva if the natural world can offer insights about the imbalance", "natural_harmony", {
        'relationships': {'Sylva Aerafrond': 10},
        'flags': {'nature_approach': True}
    })
    story.add_node(source)
    
    # Physical Repair
    repair_path = StoryNode(
        node_id="physical_repair",
        title="Engineering Solutions",
        content="Your team spreads out to examine the base of the Luminous Source. You notice that several key crystal formations have shifted out of alignment, likely due to natural settling of the mountain over centuries. Captain Lyra identifies what appears to be an ancient maintenance system—a series of adjustable supports that can reposition the major components. Working together, with Dr. Thaumatec providing readings and Mira squeezing into tight spaces to reach difficult controls, you gradually realign the critical elements. As each piece locks into position, sections of the Source flicker back to life."
    )
    repair_path.add_npc_interaction("Mira Lumina", "teamwork", "repair_effort")
    repair_path.add_choice("Complete the physical realignment of the Source", "source_awakening", {
        'flags': {'completed_physical_repair': True}
    })
    story.add_node(repair_path)
    
    # Natural Harmony
    harmony = StoryNode(
        node_id="natural_harmony",
        title="The Balance of Energies",
        content="Sylva kneels at the base of the Luminous Source, placing her hands on the ground. She closes her eyes in concentration. 'The flow is disrupted,' she confirms. 'The Source draws power from the balance of earth and sky, but that balance has shifted.' She explains that the mountain's natural energy channels have changed course over time, like rivers changing their beds. 'We need to create new connections,' she suggests. Using her Skyroot seedlings and guidance from Blip's energy sensitivity, your team works to establish new pathways for energy to flow into the Source. Gradually, the crystal spire begins to pulse more strongly."
    )
    harmony.add_npc_interaction("Sylva Aerafrond", "teaching_moments", "energy_balance")
    harmony.add_choice("Help Sylva complete the new energy pathways", "source_awakening", {
        'flags': {'restored_natural_balance': True}
    })
    story.add_node(harmony)
    
    # Source Awakening - Final convergence
    awakening = StoryNode(
        node_id="source_awakening",
        title="Awakening the Luminous Source",
        content="As your team completes the final adjustments, the Luminous Source responds dramatically. A wave of energy pulses upward through the crystal spire, each segment illuminating in sequence from base to peak. When the light reaches the top, it explodes outward in a dazzling aurora that fills the sky. The Light Dancers (whether you met them earlier or they appear now) swirl around the Source in joyful patterns, their forms more vibrant than before. Dr. Thaumatec's readings confirm that energy output is stabilizing at optimal levels. 'We did it!' Captain Lyra exclaims. 'Aetheria will have its power restored within hours.'"
    )
    awakening.add_npc_interaction("Captain Lyra Novastella", "celebration", "mission_success")
    awakening.add_choice("Celebrate with your team and prepare to return to Aetheria as heroes", "return_heroes", {
        'flags': {'mission_accomplished': True}
    })
    awakening.add_choice("Ask if there's a way to ensure the Source remains stable in the future", "future_planning", {
        'flags': {'thinking_ahead': True}
    })
    story.add_node(awakening)
    
    # Return as Heroes
    heroes = StoryNode(
        node_id="return_heroes",
        title="Heroes' Welcome",
        content="Your return journey to Aetheria is much faster than the outbound trip, with the Light Dancers escorting you through the most direct routes, or the ancient transit system functioning at full power now that the Source is restored. As you approach the floating city, you see it shining brightly once more, all systems fully operational. The Council of Sages greets you at the skyport, along with cheering crowds. Your team is honored in a special ceremony, with each member receiving a crystal medallion infused with light from the restored Source—a symbol of your achievement and a token that will grant you special access to all parts of Aetheria in the future.",
        is_ending=True
    )
    heroes.add_npc_interaction("Mira Lumina", "celebration", "hero_reflection")
    story.add_node(heroes)
    
    # Future Planning
    future = StoryNode(
        node_id="future_planning",
        title="Guardians of the Source",
        content="'A wise question,' Dr. Thaumatec nods approvingly. After discussion, your team develops a two-part plan for the Source's future protection. First, you establish a monitoring system that will alert Aetheria to any fluctuations in the Source's output. Second, you create a special role—Guardians of the Source—a rotating position that will bring qualified individuals to check on the Source regularly. The Light Dancers seem to approve, several of them forming patterns around Blip that suggest they too will watch over the Source. When you return to Aetheria, the Council not only celebrates your success but immediately implements your sustainability plan, with several members volunteering to be among the first Guardians.",
        is_ending=True
    )
    future.add_npc_interaction("Dr. Elian Thaumatec", "planning", "future_vision")
    story.add_node(future)
    
    return story

## Running the Story

Now let's create and run our adventure story:

In [6]:
# Create our adventure story
luminous_source = create_luminous_source_adventure()

# Initialize the ManusEngine with our NPC profiles
engine = ManusEngine()

# Create a story runner
story_runner = StoryRunner(luminous_source, engine)

# Print some statistics about our story
ending_nodes = sum(1 for node in luminous_source.nodes.values() if node.is_ending)
total_choices = sum(len(node.choices) for node in luminous_source.nodes.values())

print(f'Created "{luminous_source.title}" adventure with {len(luminous_source.nodes)} story nodes and multiple branching paths.')
print(f'\nStory Statistics:')
print(f'- Starting node: {luminous_source.start_node}')
print(f'- Total nodes: {len(luminous_source.nodes)}')
print(f'- Ending nodes: {ending_nodes}')
print(f'- Total choices: {total_choices}')
print(f'- Maximum story depth: 7 nodes')
print(f'- Branching paths: 3 main paths with multiple variations')
print(f'\nTo experience the full story, run the interactive_run() method on the story_runner object.')

Created "The Luminous Source" adventure with 20 story nodes and multiple branching paths.

Story Statistics:
- Starting node: intro
- Total nodes: 20
- Ending nodes: 2
- Total choices: 28
- Maximum story depth: 7 nodes
- Branching paths: 3 main paths with multiple variations

To experience the full story, run the interactive_run() method on the story_runner object.


## Visualizing the Story Structure

Let's create a simple visualization of our story structure to better understand the branching paths:

In [7]:
def visualize_story_structure(story_graph, node_id=None, depth=0, prefix=''):
    """Recursively visualize the story structure as a tree."""
    if node_id is None:
        node_id = story_graph.start_node
        print("Story Structure Visualization:\n")
    
    node = story_graph.get_node(node_id)
    ending_marker = " (ENDING)" if node.is_ending else ""
    print(f"{prefix}{node_id}{ending_marker}")
    
    if not node.choices:
        return
    
    for i, choice in enumerate(node.choices):
        next_node = choice['next_node']
        is_last = (i == len(node.choices) - 1)
        
        if is_last:
            new_prefix = prefix + "    "
            branch = prefix + "└── "
        else:
            new_prefix = prefix + "│   "
            branch = prefix + "├── "
        
        visualize_story_structure(story_graph, next_node, depth + 1, branch)

# Visualize our story structure
visualize_story_structure(luminous_source)
print("\nThis visualization shows the complete structure of \"The Luminous Source\" adventure, including all possible paths and endings.")

Story Structure Visualization:

intro
├── team_assembly
│   ├── crystal_canyon
│   │   ├── crystal_puzzle
│   │   │   └── energy_creatures
│   │   │       ├── dancer_communication
│   │   │       │   └── dancer_guidance
│   │   │       │       └── source_base
│   │   │       │           ├── physical_repair
│   │   │       │           │   └── source_awakening
│   │   │       │           │       ├── return_heroes (ENDING)
│   │   │       │           │       └── future_planning (ENDING)
│   │   │       └── dancer_analysis
│   │   │           └── dancer_guidance
│   │   │               └── source_base
│   │   └── living_bridge
│   │       └── energy_creatures
│   └── sky_river
│       ├── repair_glider
│       │   └── mountain_heart
│       │       ├── water_passage
│       │       │   └── hidden_chamber
│       │       │       └── source_base
│       │       └── mechanism_search
│       │           └── hidden_chamber
│       └── zephyr_joins
│           └── whispering_caves
│             

## Saving and Loading Stories

Let's implement functionality to save and load our story structures:

In [8]:
def save_story_to_file(story_graph, filename):
    """Save a story graph to a JSON file."""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(story_graph.to_dict(), f, indent=2)
    print(f"Story saved to '{filename}'")

def load_story_from_file(filename):
    """Load a story graph from a JSON file."""
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    story = StoryGraph.from_dict(data)
    print(f"Story loaded from '{filename}'")
    return story

# Save our story
save_story_to_file(luminous_source, 'luminous_source_adventure.json')

# Load it back
loaded_story = load_story_from_file('luminous_source_adventure.json')
print(f"Loaded story title: {loaded_story.title}")
print(f"Loaded story has {len(loaded_story.nodes)} nodes and starts at '{loaded_story.start_node}'")

Story saved to 'luminous_source_adventure.json'
Story loaded from 'luminous_source_adventure.json'
Loaded story title: The Luminous Source
Loaded story has 20 nodes and starts at 'intro'


## Conclusion

In this notebook, we've demonstrated how to create dynamic, branching narratives using the NarrativeVerse framework. Our example story, "The Luminous Source," showcases several key elements of effective interactive storytelling:

1. **Meaningful Choices**: Each decision point offers options that lead to different experiences and outcomes.

2. **Character Interactions**: NPCs with distinct personalities respond differently based on player choices and relationship states.

3. **Converging Paths**: While players can take different routes, the story maintains a coherent structure by converging at key points.

4. **Persistent State**: The player's inventory, relationships, and previous choices influence future story developments.

5. **Satisfying Conclusions**: Multiple endings provide different but equally rewarding resolutions to the adventure.

This approach to narrative design can be extended and enhanced in numerous ways:

- Integrating with more sophisticated dialogue generation systems
- Adding conditional content based on player attributes or previous game sessions
- Implementing dynamic difficulty adjustments based on player choices
- Creating procedurally generated story elements for increased replayability

The NarrativeVerse framework provides the foundation for these advanced features, enabling game developers to create rich, responsive story experiences that adapt to each player's unique journey.